[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/semantic-router/blob/main/docs/encoders/google.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/aurelio-labs/semantic-router/blob/main/docs/encoders/google.ipynb)

# Using GoogleEncoder

Google's [Pathways Language Model](https://blog.research.google/2022/04/pathways-language-model-palm-scaling-to.html) (PaLM) is a dense decoder-only model that is trained on a large corpus of text data. The hidden states of the model can be used as embeddings for text data, and Google has released versions of those layers for public use. This notebook demonstrates how to use the GoogleEncoder with the Semantic Router.

## Getting Started

We start by installing semantic-router. Support for the new `GoogleEncoder` class was added in `semantic-router==0.0.31`.

In [ ]:
!pip install -qU "semantic-router[google]"

We start by defining a dictionary mapping routes to example phrases that should trigger those routes.

In [ ]:
from semantic_router import Route

politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president",
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

Let's define another for good measure:

In [ ]:
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

routes = [politics, chitchat]

Now we initialize our embedding model. To do this with GoogleEncoder, you'll need to have an active Google Cloud Platform account and a project with the Embeddings API enabled. You can find more information on how to set up a development project in the [Google Cloud documentation](https://cloud.google.com/vertex-ai/generative-ai/docs/start/quickstarts/quickstart-text-embeddings).

In [ ]:
from semantic_router.encoders import GoogleEncoder

PROJECT_ID = "your-project-id"
LOCATION = "us-central1"

encoder = GoogleEncoder(project_id=PROJECT_ID, location=LOCATION)

Now we define the `RouteLayer`. When called, the route layer will consume text (a query) and output the category (`Route`) it belongs to — to initialize a `RouteLayer` we need our `encoder` model and a list of `routes`.

In [ ]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes)

Now we can test it:

In [ ]:
rl("don't you love politics?")

In [ ]:
rl("how's the weather today?")

Both are classified accurately, what if we send a query that is unrelated to our existing `Route` objects?

In [ ]:
rl("I'm interested in learning about llama 2")

In this case, we return `None` because no matches were identified.